---
title: "Homework 2"
format: html
---


__Due Date:__ 2022-10-16 at 8:30 AM PT

---


__Name:__ Sabahat



For this assignment, you will practice downloadings, cleaning, and analyzing data from the [National Risk Index (NRI)](https://hazards.fema.gov/nri/) and the [CDC Social Vulnerability Index (SVI)](https://www.atsdr.cdc.gov/placeandhealth/svi/index.html).

## Preparation

1. Create a 'data' folder in the root directory of your repository.
1. Inside the 'data' folder, create a 'raw' folder.
1. Add and commit a '.gitignore' file to the root directory of this repository that excludes all contents of the 'data' folder.
1. Download the county-level NRI and SVI data for the entire United States. Place the data in the 'data/raw' folder.
1. In the repository README, provide a brief (1-2 sentence) description of each file in the 'data' folder and a link to the original source of the data.

## Task 1 - NRI Data Cleaning

__1. Import the NRI data. Ensure that the [FIPS code](https://en.wikipedia.org/wiki/Federal_Information_Processing_Standard_state_code) variable ('STCOFIPS') is correctly identified as a string / character variable. Otherwise, the leading zeros will be removed.__

In [ ]:
import pandas as pd
NRI_Table_Counties = pd.read_csv('C:/Users/sabahat/Desktop/ML-Folder/Sabahat-PRGS-Intro-to-ML-2024/Homework/data/raw/NRI_Table_Counties.csv', dtype={'STCOFIPS': str})
print(NRI_Table_Counties.head())

NRI_subset = NRI_Table_Counties.filter(regex='STCOFIPS|_AFREQ$|_RISKR$')
print(NRI_subset.head())
missing_values = NRI_subset.isnull().sum()
missing_values = missing_values[missing_values.index.str.endswith(('_AFREQ', '_RISKR'))]
missing_values_df = missing_values.reset_index()
missing_values_df.columns = ['Column', 'MissingCount']
print(missing_values_df)
NRI_Table_Counties['AVLN_AFREQ_Missing'] = NRI_Table_Counties['AVLN_AFREQ'].isnull()
cross_tab = pd.crosstab(NRI_Table_Counties['AVLN_AFREQ_Missing'], NRI_Table_Counties['AVLN_RISKR'], dropna=False)
print(cross_tab)
afreq_columns = [col for col in NRI_Table_Counties.columns if col.endswith('_AFREQ')]
for afreq_col in afreq_columns:
    risk_col = afreq_col.replace('_AFREQ', '_RISKR')
    NRI_Table_Counties.loc[NRI_Table_Counties[risk_col] == 'Not Applicable', afreq_col] = NRI_Table_Counties[afreq_col].fillna(0)
    print(NRI_Table_Counties[afreq_columns].head())

    ## Task 2 - SVI Data Cleaning
SVI_2022_US_county= pd.read_csv('C:/Users/sabahat/Desktop/ML-Folder/Sabahat-PRGS-Intro-to-ML-2024/Homework/data/raw/SVI_2022_US_county.csv', dtype={'FIPS': str})
columns_to_keep = [
    'ST', 'STATE', 'ST_ABBR', 'STCNTY', 'COUNTY', 'FIPS', 'LOCATION', 
    'AREA_SQMI', 'E_TOTPOP', 'EP_POV150', 'EP_UNEMP', 'EP_HBURD', 
    'EP_NOHSDP', 'EP_UNINSUR', 'EP_AGE65', 'EP_AGE17', 'EP_DISABL', 
    'EP_SNGPNT', 'EP_LIMENG', 'EP_MINRTY', 'EP_MUNIT', 'EP_MOBILE', 
    'EP_CROWD', 'EP_NOVEH', 'EP_GROUPQ', 'EP_NOINT', 'EP_AFAM', 
    'EP_HISP', 'EP_ASIAN', 'EP_AIAN', 'EP_NHPI', 'EP_TWOMORE', 'EP_OTHERRACE']
svi_subset = SVI_2022_US_county[columns_to_keep]
print(svi_subset.head())
missing_values = SVI_2022_US_county.isnull().sum()
missing_values_df = missing_values.reset_index()
missing_values_df.columns = ['Column', 'MissingCount']
print(missing_values_df)
# ======  Task 3 - Data Merging ======
nri_fips = set(NRI_Table_Counties['STCOFIPS'])
svi_fips = set(SVI_2022_US_county['FIPS'])
nri_not_in_svi = nri_fips - svi_fips
svi_not_in_nri = svi_fips - nri_fips
print("FIPS codes in NRI but not in SVI:", nri_not_in_svi)
print("FIPS codes in SVI but not in NRI:", svi_not_in_nri)
merged_data = pd.merge(NRI_Table_Counties, SVI_2022_US_county, left_on='STCOFIPS', right_on='FIPS', how='outer')
print(merged_data.head())
missing_values = merged_data.isnull().sum()
print(missing_values_df)
## Task 4 - Data Analysis
def plot_histogram(column):
    plt.figure(figsize=(8, 6))
    plt.hist(merged_data[column].dropna(), bins=30, alpha=0.7, color='blue')
    plt.title(f'Histogram of {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()
    numerical_columns = merged_data.select_dtypes(include=['float64', 'int64']).columns
for column in numerical_columns:
    plot_histogram(column)